In [1]:
# %pip install cplex, docplex, pandas, tensorflow

In [2]:
import os
import cplex
import pandas as pd 
from time import time
import tensorflow as tf
import docplex.mp.model as mp
from docplex.mp import model_reader
from typing import List, Tuple, Any
from docplex.mp.constr import LinearConstraint 
from utils.explanations import get_minimal_explanation, get_explanation_relaxed

Explicar instancia

In [3]:
# @title explain_instance
def explain_instance(
    initial_network,
    configuration: dict,
    instance_index: int,
    data: pd.DataFrame,
    model_h5,
    n_classes
) -> Tuple[List[LinearConstraint], mp.Model, int]:
    method = configuration["method"]
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables,
        bounds,
    ) = initial_network
    network_input = data.iloc[instance_index, :-1]
    # print(network_input)  # network_input = instance
    network_input = tf.reshape(tf.constant(network_input), (1, -1))
    network_output = model_h5.predict(tf.constant(network_input))[0]
    network_output = tf.argmax(network_output)
    mdl_aux = mdl_milp_with_binary_variable.clone()
    (explanation, model) = get_minimal_explanation(
        mdl_aux,
        network_input,
        network_output,
        n_classes=n_classes,
        method=method,
        output_bounds=output_bounds_binary_variables
    )

    return (explanation, model, network_output)

In [4]:
# @title explain_instance_relaxed
def explain_instance_relaxed(
    initial_network,
    initial_network_relaxed,
    configuration: dict,
    instance_index: int,
    data: pd.DataFrame,
    model_h5,
    n_classes,
    delta=1.0
) -> Tuple[List[LinearConstraint], mp.Model]:
    method = configuration["method"]
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables,
        bounds,
    ) = initial_network

    model_milp_relaxed, output_bounds_relaxed = initial_network_relaxed
    network_input = data.iloc[instance_index, :-1]
    # print(network_input)  # network_input = instance
    network_input = tf.reshape(tf.constant(network_input), (1, -1))
    network_output = model_h5.predict(tf.constant(network_input))[0]
    network_output = tf.argmax(network_output)

    mdl_aux = model_milp_relaxed.clone()

    (explanation, model) = get_explanation_relaxed(
        mdl_aux,
        network_input,
        network_output,
        n_classes=n_classes,
        # method=method,
        # output_bounds=output_bounds_binary_variables,
        delta=delta,
        # min_max_results_path_relaxed_global = min_max_results_path_relaxed_
    )

    return (explanation, model)

Utils

In [5]:
def export_milp_as_lp(mdl: mp.Model, file: str):
  mdl.export_as_lp(f"{file}")

def read_cplex_model(file: str):
  return cplex.Cplex(file)

def convert_string_to_pixel(pixel_str: str, matrix_size: tuple[int, int]) -> tuple[int, int]:
    # Remover o prefixo "x_"
    pixel_number = int(pixel_str.split("_")[1])
    # Obter as dimensões da matriz
    rows, cols = matrix_size
    # Calcular as coordenadas do pixel
    row_index = pixel_number // cols
    col_index = pixel_number % cols
    return row_index, col_index
  
def get_coordinates_from_explanation(
  explanation: list[LinearConstraint],
  matrix_size: tuple[int, int],
) -> list[tuple[int, int]]:
    coordinates = []
    for constraint in explanation:
        # Extrair coordenadas da variável associada à restrição linear
        variable_name = constraint.left_expr.name
        x, y = convert_string_to_pixel(variable_name, matrix_size)
        coordinates.append((x, y))
    return coordinates

def create_directory(path_to_create:str):
  array_splited = path_to_create.split('/')
  min_max_path_full = './'
  for path in array_splited:
    if path == '':
      continue
    min_max_path_full = f'{min_max_path_full}/{path}'
    if not os.path.exists(min_max_path_full):
      os.makedirs(min_max_path_full)
  return array_splited

Benchmark

In [6]:

def benchmark_instance( 
    initial_network: Any,
    initial_network_relaxed: Any,
    model_h5: Any,
    n_classes: int,
    data: pd.DataFrame,
    instance_index: int,
    resultados: pd.DataFrame,
    file_result:str,
    delta = 0.1):
    
    method = "fischetti"

    network_input = data.iloc[instance_index, :-1]
    network_input = tf.reshape(tf.constant(network_input), (1, -1))
    network_output = model_h5.predict(tf.constant(network_input))[0]
    network_output = tf.argmax(network_output)
    
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables, 
    ) = initial_network
    
    # Relaxed
    model_milp_relaxed, output_bounds_relaxed = initial_network_relaxed
    mdl_aux_2 = model_milp_relaxed.clone()
    start_time = time()
    (explanation_relaxed_global, model_relaxed_global) = get_explanation_relaxed(
        mdl_aux_2,
        network_input,
        network_output,
        n_classes=n_classes,
        delta=1,
    )
    end_time = time()
    time_relaxed_global = end_time - start_time
    len_relaxed_global = len(explanation_relaxed_global)
    
    # Relaxed Local
    model_milp_relaxed_local, output_bounds_relaxed = initial_network_relaxed
    mdl_aux_2 = model_milp_relaxed_local.clone()
    start_time = time()
    (explanation_relaxed_local, model_relaxed_local) = get_explanation_relaxed(
        mdl_aux_2,
        network_input,
        network_output,
        n_classes=n_classes,
        delta=delta,
    )
    end_time = time()
    time_relaxed_local = end_time - start_time
    len_relaxed_local = len(explanation_relaxed_local)
    
    # Original
    mdl_aux = mdl_milp_with_binary_variable.clone()
    start_time = time()
    (explanation, model) = get_minimal_explanation(
       mdl_aux,
       network_input,
       network_output,
       n_classes=n_classes,
       method=method,
       output_bounds=output_bounds_binary_variables,
    )
    end_time = time()
    tempo_original = end_time - start_time
    len_original = len(explanation)
    
    len_resultados = len(resultados)
    resultados.loc[len_resultados] = [
        instance_index,
        
        tempo_original,
        time_relaxed_local,
        time_relaxed_global,
        
        len_original, 
        len_relaxed_local,
        len_relaxed_global,
        
        delta,
        
        explanation, 
        explanation_relaxed_local,
        explanation_relaxed_global, 
    ]
    resultados.to_csv(f'{file_result}', index=False)

Datasets

In [7]:
from dataclasses import dataclass
datasets_path = "./datasets"
@dataclass
class Dataset:
    dir_path: str
    model: str
    n_classes: int

datasets: List[Dataset] = [
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_1layers_20neurons.h5",
        n_classes=10,
    ),
    Dataset(
        dir_path=f"{datasets_path}/digits",
        model="models/model_2layers_20neurons.h5",
        n_classes=10,
    ),
    
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_1layers_20neurons.h5",
        n_classes=3,
    ),
    Dataset(
        dir_path=f"{datasets_path}/iris",
        model="models/model_2layers_20neurons.h5",
        n_classes=3,
    ),  
    Dataset(
        dir_path=f"{datasets_path}/breast_cancer",
        model="models/model_1layers_20neurons.h5",
        n_classes=2,
    ),  
    Dataset(
        dir_path=f"{datasets_path}/breast_cancer",
        model="models/model_2layers_20neurons.h5",
        n_classes=2,
    ),  
]

configurations = [{"method": "fischetti", "relaxe_constraints": True}]

In [8]:
def get_data(str_df):
    data_train = pd.read_csv(f"{str_df}/train.csv")
    data_test = pd.read_csv(f"{str_df}/test.csv")
    return pd.concat([data_train, data_test])

dataset_index = 7
dataset = datasets[dataset_index]
dir_path = dataset.dir_path
n_classes = dataset.n_classes
model_h5_file = dataset.model
model_h5 = tf.keras.models.load_model(f"{dir_path}/{dataset.model}")
data = get_data(dir_path)

In [9]:
def create_results_directory(dir_path:str, model_h5_file:str)->str:
  file_name_model = (f"{dir_path}/{model_h5_file}")
  file_name_model_result = (f"{dir_path}/results/{model_h5_file}")
  if not os.path.exists(file_name_model_result):
    os.makedirs(file_name_model_result)
  return file_name_model_result

path_result = create_results_directory(dir_path, model_h5_file)

In [10]:
path_result

'./datasets/breast_cancer/results/models/model_2layers_20neurons.h5'

In [11]:
from utils.milp import codify_network, codify_network_relaxed

initial_network = codify_network(model_h5, data, "fischetti", True)
(
    mdl_milp_with_binary_variable,
    output_bounds_binary_variables,
    # bounds,
) = initial_network

initial_network_relaxed = codify_network_relaxed(
    model_h5,
    mdl_milp_with_binary_variable,
    data,
    "fischetti",
    True,
    output_bounds_binary_variables
)

In [12]:
instance_index = 1
delta = 0.47

In [13]:
# @title criar dataframe dos resultados

file_results = f"{path_result}/df.csv"
if os.path.exists(file_results):
    resultados = pd.read_csv(file_results)
else:
    resultados = pd.DataFrame(
        columns=[
            "instance_index",
            "tempo_original",
            "tempo_relaxado",
            "tempo_relaxado_global",
            "len_original",
            "len_relaxado",
            "len_relaxado_global",
            "delta",
            "explanation",
            "explanation_relaxed",
            "explanation_relaxed_global",
        ]
    )

In [14]:
# r = benchmark_instance( 
#     initial_network = initial_network,
#     initial_network_relaxed = initial_network_relaxed,
#     model_h5 = model_h5,
#     n_classes = n_classes,
#     data = data,
#     instance_index = instance_index,
#     resultados = resultados,
#     file_result = file_results,
#     delta = delta
# )

In [15]:
inicio = 382
fim = len(data)
for i in range(inicio, fim):
    r = benchmark_instance( 
        initial_network = initial_network,
        initial_network_relaxed = initial_network_relaxed,
        model_h5 = model_h5,
        n_classes = n_classes,
        data = data,
        instance_index = i,
        resultados = resultados,
        file_result = file_results,
        delta = delta
    )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var(type=C,name='o_1',lb=-9.67127,ub=22.597)]
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var(type=C,name='o_1',lb=-9.67127,ub=22.597)]
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var(type=C,name='o_1',lb=-9.67127,ub=22.597)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var(type=C,name='o_1',lb=-9.67127,ub=22.597)]
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var(type=C,name='o_1',lb=-9.67127,ub=22.597)]
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var(type=C,name='o_1',lb=-9.67127,ub=22.597)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var(type=C,name='o_1',lb=-9.67127,ub=22.597)]
[docplex.mp.Var(type=C,name='o_0',lb=-24.0941,ub=15.5793), docplex.mp.Var

In [ ]:
import random

def experiments_with_md():
    for dataset in datasets:
        dir_path = dataset.dir_path
        n_classes = dataset.n_classes
        model_h5_file = dataset.model
        model_h5 = tf.keras.models.load_model(f"{dir_path}/{dataset.model}")
        data = get_data(dir_path)
        path_result = create_results_directory(dir_path, model_h5_file)
        initial_network = codify_network(model_h5, data, "fischetti", True)
        (
            mdl_milp_with_binary_variable,
            output_bounds_binary_variables,
            # bounds,
        ) = initial_network

        initial_network_relaxed = codify_network_relaxed(
            model_h5,
            mdl_milp_with_binary_variable,
            data,
            "fischetti",
            True,
            output_bounds_binary_variables
        )

        file_results = f"{path_result}/df.csv"
        if os.path.exists(file_results):
            resultados = pd.read_csv(file_results)
        else:
            resultados = pd.DataFrame(
                columns=[
                    "instance_index",
                    "tempo_original",
                    "tempo_relaxado",
                    "tempo_relaxado_global",
                    "len_original",
                    "len_relaxado",
                    "len_relaxado_global",
                    "delta",
                    "explanation",
                    "explanation_relaxed",
                    "explanation_relaxed_global",
                ]
            )

    
    
        qtd_instancias = 100
        instancias =  random.sample(range(0, len(data)), qtd_instancias) # seleciona 100 instancias aleatorias
        deltas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        for instancia_indice in instancias:
            r = benchmark_instance( 
                initial_network = initial_network,
                initial_network_relaxed = initial_network_relaxed,
                model_h5 = model_h5,
                n_classes = n_classes,
                data = data,
                instance_index = instancia_indice,
                resultados = resultados,
                file_result = file_results,
                delta = delta
            )
        